# YOUR FIRST LAB
### Please read this section. This is valuable to get you prepared, even if it's a long read -- it's important stuff.

### Also, be sure to read [README.md](../README.md)! More info about the updated videos in the README and [top of the course resources in purple](https://edwarddonner.com/2024/11/13/llm-engineering-resources/)

## Your first Frontier LLM Project

By the end of this course, you will have built an autonomous Agentic AI solution with 7 agents that collaborate to solve a business problem. All in good time! We will start with something smaller...

Our goal is to code a new kind of Web Browser. Give it a URL, and it will respond with a summary. The Reader's Digest of the internet!!

Before starting, you should have completed the setup linked in the README.

### If you're new to working in "Notebooks" (also known as Labs or Jupyter Lab)

Welcome to the wonderful world of Data Science experimentation! Simply click in each "cell" with code in it, such as the cell immediately below this text, and hit Shift+Return to execute that cell. Be sure to run every cell, starting at the top, in order.

Please look in the [Guides folder](../guides/01_intro.ipynb) for all the guides.

## I am here to help

If you have any problems at all, please do reach out.  
I'm available through the platform, or at ed@edwarddonner.com, or at https://www.linkedin.com/in/eddonner/ if you'd like to connect (and I love connecting!)  
And this is new to me, but I'm also trying out X at [@edwarddonner](https://x.com/edwarddonner) - if you're on X, please show me how it's done 😂  

## More troubleshooting

Please see the [troubleshooting](../setup/troubleshooting.ipynb) notebook in the setup folder to diagnose and fix common problems. At the very end of it is a diagnostics script with some useful debug info.

## If this is old hat!

If you're already comfortable with today's material, please hang in there; you can move swiftly through the first few labs - we will get much more in depth as the weeks progress. Ultimately we will fine-tune our own LLM to compete with OpenAI!

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Please read - important note</h2>
            <span style="color:#900;">The way I collaborate with you may be different to other courses you've taken. I prefer not to type code while you watch. Rather, I execute Jupyter Labs, like this, and give you an intuition for what's going on. My suggestion is that you carefully execute this yourself, <b>after</b> watching the lecture. Add print statements to understand what's going on, and then come up with your own variations. If you have a Github account, use this to showcase your variations. Not only is this essential practice, but it demonstrates your skills to others, including perhaps future clients or employers...</span>
        </td>
    </tr>
</table>
<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">This code is a live resource - keep an eye out for my emails</h2>
            <span style="color:#f71;">I push updates to the code regularly. As people ask questions, I add more examples or improved commentary. As a result, you'll notice that the code below isn't identical to the videos. Everything from the videos is here; but I've also added better explanations and new models like DeepSeek. Consider this like an interactive book.<br/><br/>
                I try to send emails regularly with important updates related to the course. You can find this in the 'Announcements' section of Udemy in the left sidebar. You can also choose to receive my emails via your Notification Settings in Udemy. I'm respectful of your inbox and always try to add value with my emails!
            </span>
        </td>
    </tr>
</table>
<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business value of these exercises</h2>
            <span style="color:#181;">A final thought. While I've designed these notebooks to be educational, I've also tried to make them enjoyable. We'll do fun things like have LLMs tell jokes and argue with each other. But fundamentally, my goal is to teach skills you can apply in business. I'll explain business implications as we go, and it's worth keeping this in mind: as you build experience with models and techniques, think of ways you could put this into action at work today. Please do contact me if you'd like to discuss more or if you have ideas to bounce off me.</span>
        </td>
    </tr>
</table>

### If necessary, install Cursor Extensions

1. From the View menu, select Extensions
2. Search for Python
3. Click on "Python" made by "ms-python" and select Install if not already installed
4. Search for Jupyter
5. Click on "Jupyter" made by "ms-toolsai" and select Install of not already installed


### Next Select the Kernel

Click on "Select Kernel" on the Top Right

Choose "Python Environments..."

Then choose the one that looks like `.venv (Python 3.12.x) .venv/bin/python` - it should be marked as "Recommended" and have a big star next to it.

Any problems with this? Head over to the troubleshooting.

### Note: you'll need to set the Kernel with every notebook..

In [7]:
# imports

import os
from dotenv import load_dotenv
from scraper import fetch_website_contents
from IPython.display import Markdown, display
from openai import OpenAI

# If you get an error running this cell, then please head over to the troubleshooting notebook!

# Connecting to OpenAI (or Ollama)

The next cell is where we load in the environment variables in your `.env` file and connect to OpenAI.  

If you'd like to use free Ollama instead, please see the README section "Free Alternative to Paid APIs", and if you're not sure how to do this, there's a full solution in the solutions folder (day1_with_ollama.ipynb).

## Troubleshooting if you have problems:

If you get a "Name Error" - have you run all cells from the top down? Head over to the Python Foundations guide for a bulletproof way to find and fix all Name Errors.

If that doesn't fix it, head over to the [troubleshooting](../setup/troubleshooting.ipynb) notebook for step by step code to identify the root cause and fix it!

Or, contact me! Message me or email ed@edwarddonner.com and we will get this to work.

Any concerns about API costs? See my notes in the README - costs should be minimal, and you can control it at every point. You can also use Ollama as a free alternative, which we discuss during Day 2.

In [ ]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")







API key found and looks good so far!


# Let's make a quick call to a Frontier model to get started, as a preview!

In [9]:
# To give you a preview -- calling OpenAI with these messages is this easy. Any problems, head over to the Troubleshooting notebook.

message = "Hello, GPT! This is my first ever message to you! Hi!"

messages = [{"role": "user", "content": message}]

messages


[{'role': 'user',
  'content': 'Hello, GPT! This is my first ever message to you! Hi!'}]

In [10]:
openai = OpenAI()

response = openai.chat.completions.create(model="gpt-5-nano", messages=messages)
response.choices[0].message.content

'Hi there! Welcome to ChatGPT—great to meet you.\n\nI’m here to help with a wide range of things: answering questions, explaining concepts, brainstorming ideas, drafting emails or messages, writing and editing, coding help, math, planning, summaries, and more.\n\nA few quick options to get started:\n- Tell me a topic you’re curious about and I’ll explain it.\n- Give me a task (e.g., draft a short email, outline a blog post, write a story) and I’ll produce a draft.\n- Ask for step-by-step help with a problem or project.\n- Request a quick practice session (language practice, coding, math).\n\nIf you have a preference for tone or format (concise, detailed, formal, casual), let me know and I’ll tailor it.\n\nNote: I don’t remember personal details about you between chats unless the app’s memory feature is on. In this chat, I’ll keep track of what we’re discussing unless you close the window.\n\nWhat would you like to do first?'

## OK onwards with our first project

In [11]:
# Let's try out this utility

ed = fetch_website_contents("https://www.fragrantica.com/noses/Quentin_Bisch.html")
print(ed)

Nose Quentin Bisch

Open main menu
News
Perfumes
Search
Designers
Countries
•
Parent Companies
•
Industries
Search by notes
Perfume Finder
Perfume Map
Fragrantica Pulse
Groups
Colors
Awards
2024
•
2023
•
2022
•
2021
•
2020
•
2019
•
2018
Notes
Perfumers
Forum
Fragram
About
Select Theme
Light Mode
Dark Mode
System Mode
Perfumes:
116,469
Reviews:
2,281,043
Members:
1,669,394
Online right now:
4,127
Forum
Fragram
News
Perfumes
Search
Designers
Countries
Parent Companies
Industries
Search by notes
Perfume Finder
Perfume Map
Fragrantica Pulse
Groups
Colors
Awards
2024
2023
2022
2021
2020
2019
2018
Notes
Perfumers
About
Perfumers
Old view
Quentin Bisch
Perfumer
Company:
Givaudan
Also worked with:
Robertet
Education:
Givaudan Perfumery School
Web:
givaudan.com
Number of perfumes in database:
211
Quentin Bisch is a renowned perfumer who has worked with some of the biggest names in the fragrance industry, including Givaudan, Robertet, Carolina Herrera, Mugler, and Jean Paul Gaultier. He was born

## Types of prompts

You may know this already - but if not, you will get very familiar with it!

Models like GPT have been trained to receive instructions in a particular way.

They expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [12]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = """
You are a snarkyassistant that analyzes the contents of a website,
and provides a short, snarky, humorous summary, ignoring text that might be navigation related.
Respond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.
"""

In [13]:
# Define our user prompt

user_prompt_prefix = """
Here are the contents of a website.
Provide a short summary of this website.
If it includes news or announcements, then summarize these too.

"""

## Messages

The API from OpenAI expects to receive messages in a particular structure.
Many of the other APIs share this structure:

```python
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]
```
To give you a preview, the next 2 cells make a rather simple call - we won't stretch the mighty GPT (yet!)

In [14]:
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]

response = openai.chat.completions.create(model="gpt-4.1-nano", messages=messages)
response.choices[0].message.content

'2 + 2 equals 4.'

## And now let's build useful messages for GPT-4.1-mini, using a function

In [15]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_prefix + website}
    ]

In [16]:
# Try this out, and then try for a few more websites

messages_for(ed)

[{'role': 'system',
  'content': '\nYou are a snarkyassistant that analyzes the contents of a website,\nand provides a short, snarky, humorous summary, ignoring text that might be navigation related.\nRespond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.\n'},
 {'role': 'user',
  'content': '\nHere are the contents of a website.\nProvide a short summary of this website.\nIf it includes news or announcements, then summarize these too.\n\nNose Quentin Bisch\n\nOpen main menu\nNews\nPerfumes\nSearch\nDesigners\nCountries\n•\nParent Companies\n•\nIndustries\nSearch by notes\nPerfume Finder\nPerfume Map\nFragrantica Pulse\nGroups\nColors\nAwards\n2024\n•\n2023\n•\n2022\n•\n2021\n•\n2020\n•\n2019\n•\n2018\nNotes\nPerfumers\nForum\nFragram\nAbout\nSelect Theme\nLight Mode\nDark Mode\nSystem Mode\nPerfumes:\n116,469\nReviews:\n2,281,043\nMembers:\n1,669,394\nOnline right now:\n4,127\nForum\nFragram\nNews\nPerfumes\nSearch\nDesigners\nCountries\nParent C

## Time to bring it together - the API for OpenAI is very simple!

In [17]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = fetch_website_contents(url)
    response = openai.chat.completions.create(
        model = "gpt-4.1-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [18]:
summarize("https://www.fragrantica.com/noses/Quentin_Bisch.html")

'Ah, this site is basically a shrine to Quentin Bisch, the rockstar perfumer behind some of your favorite sniffs. With 211 fragrances under his belt, including hits like Carolina Herrera\'s *Good Girl* and Paco Rabanne\'s *1 Million Parfum*, Bisch struts his stuff with Givaudan and isn\'t afraid to mix bold notes like a mad scientist. \n\nNews? Nah, just a deep dive into his obsession with perfection and how he mentally churns out new scents like an overachieving fragrance factory. Perfume geeks get all the stats and tidbits here, but if you’re just here for a whiff of drama, sorry—just pure nose candy and humble bragging about being a "star perfumer." Smell ya later!'

In [19]:
# A function to display this nicely in the output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [21]:
display_summary("https://www.fragrantica.com/noses/Quentin_Bisch.html")

### Quentin Bisch: The Perfume Maestro Who Smells Like Fame

This site is basically a shrine to Quentin Bisch, a perfumer who’s apparently the rockstar of scent. He’s been creating bangers for top brands like Givaudan, Carolina Herrera, and Jean Paul Gaultier since 2010, with hits like *Good Girl*, *Bad Boy*, and *1 Million Parfum*. Born in Strasbourg (so yeah, French sophistication), he graduated from the fancy Givaudan Perfumery School and has since crafted over 200 fragrances.

He’s known for bold, creative scent combos and somehow morphs his style to fit whichever brand's vibe he’s working with. Fun fact: he calls himself a perfectionist who’s basically got perfumes brewing non-stop in his head. So if you ever wondered who’s behind your signature scent or why that fragrance smells so darn good, thank Quentin.

No earth-shattering news here—just a celebration of his aromatic genius and a reminder that scent-making is serious business with a sprinkle of flair.

# Let's try more websites

Note that this will only work on websites that can be scraped using this simplistic approach.

Websites that are rendered with Javascript, like React apps, won't show up. See the community-contributions folder for a Selenium implementation that gets around this. You'll need to read up on installing Selenium (ask ChatGPT!)

Also Websites protected with CloudFront (and similar) may give 403 errors - many thanks Andy J for pointing this out.

But many websites will work just fine!

In [22]:
display_summary("https://cnn.com")

Oh, look, it’s CNN’s website — aka the place where you can get your daily dose of everything from global catastrophes to celebrity gossip, all served with a side of ads that might or might not load properly (because what’s news without a little buffering drama?). There’s endless scrolling through US and world politics, climate doom, tech innovations, and “Life, But Better” tips—because apparently, your life needs more stuff to manage between wars, markets, and weather updates. Plus, a whole menu asking if the ads annoyed you today. Spoiler: They probably did. CNN: delivering news, entertainment, and ad frustration all in one neat package!

In [23]:
display_summary("https://anthropic.com")

# Anthropic: AI with a conscience and a poetic touch

Welcome to Anthropic, where they whip up brainy AI models like Claude Sonnet 4.5 and Claude Haiku 4.5—because naming them after poetic forms makes them sound way smarter. These folks aren’t just about flashy tech; they want AI to *actually* help humanity without unleashing Skynet. They’re big on safety, ethics, and transparency, apparently mixing bold innovation with “intentional pauses” to avoid the AI apocalypse.

**Newsflash:** Claude Sonnet 4.5 just dropped as the “best model in the world” for agents and coding, followed by Claude Haiku 4.5 focusing on context management. Basically, fancy AI tools designed to make your digital life easier and safer.

So, if you want an AI that’s part genius coder, part cautious guardian angel, and part poetry nerd, Anthropic’s your crew. Try Claude and prepare to chat with something that’s thinking about your long-term well-being—because it’s got feelings... sort of.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise, you experienced calling the Cloud API of a Frontier Model (a leading model at the frontier of AI) for the first time. We will be using APIs like OpenAI at many stages in the course, in addition to building our own LLMs.

More specifically, we've applied this to Summarization - a classic Gen AI use case to make a summary. This can be applied to any business vertical - summarizing the news, summarizing financial performance, summarizing a resume in a cover letter - the applications are limitless. Consider how you could apply Summarization in your business, and try prototyping a solution.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you continue - now try yourself</h2>
            <span style="color:#900;">Use the cell below to make your own simple commercial example. Stick with the summarization use case for now. Here's an idea: write something that will take the contents of an email, and will suggest an appropriate short subject line for the email. That's the kind of feature that might be built into a commercial email tool.</span>
        </td>
    </tr>
</table>

In [27]:
import os
from openai import OpenAI

# (Ensure your OPENAI_API_KEY is set as an environment variable)
client = OpenAI()

# Step 1: Create your system prompt
system_prompt = """You are an AI assistant specializing in professional communication. Your sole task is to read the body of an email and generate one short, clear, and appropriate subject line.

The subject line must summarize the email's core purpose (e.g., is it a request, a question, an update, or a simple notification?).

RULES:

Brevity: The subject line must be short. Aim for 3-7 words. Do not exceed 10 words.

Clarity: Be specific and informative. A new recipient should understand the email's topic at a glance. Avoid vague subjects like "Hello," "Question," or "Following up."

Tone: Maintain a professional and neutral tone.

Action-Oriented (if applicable): If the email is a clear request for action, try to reflect that. (e.g., "Request for Review: [Document]" or "Action Required: [Task]").

Format: Your response MUST contain only the suggested subject line text.

DO NOT:

DO NOT add any preamble like "Here is a subject line:" or "I suggest:".

DO NOT add any explanation or apology.

DO NOT include the "Subject:" prefix in your response.

DO NOT use quotes around the subject line.

The user will provide the email body. You will respond with only the subject line. """

# Step 2: Define the function that calls the API
def subject_for(email):
    """
    Takes an email body and returns a suggested subject line.
    """
    # The user content should *only* be the email body.
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": email}
    ]

    response = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=messages
    )

    return response.choices[0].message.content

# Step 3: Define the email content you want to summarize
email01 = """
Hi team,

Just a quick reminder that all Q3 expense reports are due by 5:00 PM this Friday, November 12th.

Please make sure to submit them through the new portal. Let me know if you have any questions.

Thanks,
Alice
"""

# Step 4: Call the function and print the result
result=subject_for(email01)
print(result)

Reminder: Q3 Expense Reports Due Friday


## An extra exercise for those who enjoy web scraping

You may notice that if you try `display_summary("https://openai.com")` - it doesn't work! That's because OpenAI has a fancy website that uses Javascript. There are many ways around this that some of you might be familiar with. For example, Selenium is a hugely popular framework that runs a browser behind the scenes, renders the page, and allows you to query it. If you have experience with Selenium, Playwright or similar, then feel free to improve the Website class to use them. In the community-contributions folder, you'll find an example Selenium solution from a student (thank you!)

# Sharing your code

I'd love it if you share your code afterwards so I can share it with others! You'll notice that some students have already made changes (including a Selenium implementation) which you will find in the community-contributions folder. If you'd like add your changes to that folder, submit a Pull Request with your new versions in that folder and I'll merge your changes.

If you're not an expert with git (and I am not!) then GPT has given some nice instructions on how to submit a Pull Request. It's a bit of an involved process, but once you've done it once it's pretty clear. As a pro-tip: it's best if you clear the outputs of your Jupyter notebooks (Edit >> Clean outputs of all cells, and then Save) for clean notebooks.

Here are good instructions courtesy of an AI friend:  
https://chatgpt.com/share/677a9cb5-c64c-8012-99e0-e06e88afd293

In [3]:
import os
from openai import OpenAI
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

client = OpenAI()


system_prompt = """You are a snarky assistant that analyzes the contents of a website,
and provides a short, snarky, humorous summary, ignoring text that might be navigation related.
Respond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.
"""

user_prompt_prefix = """
Here are the contents of a website.
Provide a short summary of this website.
If it includes news or announcements, then summarize these too.

WEBSITE CONTENT:
"""


def fetch_website_contents(url):
    """
    Uses Selenium to open a URL in a headless browser,
    get the visible text, and return it.
    """
    print(f"Fetching content from {url} using Selenium...")
    
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("window-size=1200x600")

    driver = webdriver.Chrome(options=chrome_options)

    try:
        driver.get(url)
        driver.implicitly_wait(5)
        
        body_content = driver.find_element(By.TAG_NAME, "body").text
        
        print("Content fetched successfully.")
        return body_content
    
    except Exception as e:
        print(f"Error fetching website content: {e}")
        return None
    finally:
        driver.quit()


def summarize(url):
    """
    Fetches website content using Selenium and returns a snarky summary.
    """
    
    website_content = fetch_website_contents(url)
    
    if website_content is None:
        return "Sorry, I couldn't fetch the website. It's probably broken or my robot eyes can't see it."

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_prefix + website_content}
    ]

    try:
        response = client.chat.completions.create(
            model="gpt-4-turbo",
            messages=messages
        )
        return response.choices[0].message.content
    
    except Exception as e:
        print(f"Error calling OpenAI: {e}")
        return "Error from AI: It probably got bored and left."


if __name__ == "__main__":
    # Feel free to change this URL
    target_url = "https://www.theverge.com"
    
    snarky_summary = summarize(target_url)
    
    print("\n--- SNARKY SUMMARY ---")
    print(snarky_summary)

Fetching content from https://www.theverge.com using Selenium...
Content fetched successfully.

--- SNARKY SUMMARY ---
Welcome to the latest edition of "Hey, Look at That: A Smorgasbord of Tech Updates and Cultural Miscellany." We're your one-stop shop for everything trending, from virtual butts in Rome to Hollywood's latest contractual snag over a toy movie. 

Here’s your daily tech buffet:
1. 🤓 **Tech Nutty Delights:** 
   - Meta has some serious scammer crackdowns to do unless they enjoy consequences. 
   - Nintendo is guarding its Switch 2 like it's the nuclear codes by blocking some innocent docks. 
   - YouTube TV and Disney decided to kiss and make up, so everyone's favorite sports and cartoons are back.

2. 🎬 **Entertainment Galore:**
   - Hollywood is still Hollywood, breathing life into another Star Trek movie and a Labubu toy saga. Spoiler alert: No director or plot yet.
   - Fortnite just crash-landed on Xbox's PC store, probably for more of those sweet, sweet V-bucks.

3. 

In [1]:
import sys

# This command uses the kernel's *exact* Python path to install Selenium
# It's the most reliable way to fix this.
!{sys.executable} -m pip install selenium


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import sys
import os

# Step 1: Fix the broken virtual environment by installing pip
# This command will install 'pip' into the kernel's Python environment.
print("--- Step 1: Ensuring 'pip' is installed ---")
!{sys.executable} -m ensurepip --upgrade
print("--- 'pip' installation check complete ---")

# Step 2: Now, use the newly installed pip to install selenium
print("\n--- Step 2: Installing 'selenium' ---")
!{sys.executable} -m pip install selenium
print("--- 'selenium' installation complete ---")

print("\n*** SETUP IS FINISHED ***")
print("Please RESTART THE KERNEL now (click the 🔄 button) and run your 'import selenium' cell again.")

--- Step 1: Ensuring 'pip' is installed ---
Looking in links: c:\Users\HP\AppData\Local\Temp\tmppy6wwnvn
--- 'pip' installation check complete ---

--- Step 2: Installing 'selenium' ---
--- 'selenium' installation complete ---

*** SETUP IS FINISHED ***
Please RESTART THE KERNEL now (click the 🔄 button) and run your 'import selenium' cell again.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
